In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df_train = pd.read_csv('TRAIN.csv',index_col = 'Date')
df_test = pd.read_csv('TEST_FINAL.csv',index_col = 'Date')

In [3]:
print(df_train.shape)
print(df_test.shape)

(188340, 9)
(22265, 7)


In [4]:
df_train.isnull().sum()

ID               0
Store_id         0
Store_Type       0
Location_Type    0
Region_Code      0
Holiday          0
Discount         0
#Order           0
Sales            0
dtype: int64

In [5]:
df_test.isnull().sum()

ID               0
Store_id         0
Store_Type       0
Location_Type    0
Region_Code      0
Holiday          0
Discount         0
dtype: int64

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188340 entries, 2018-01-01 to 2019-05-31
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  object 
 3   Location_Type  188340 non-null  object 
 4   Region_Code    188340 non-null  object 
 5   Holiday        188340 non-null  int64  
 6   Discount       188340 non-null  object 
 7   #Order         188340 non-null  int64  
 8   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(5)
memory usage: 14.4+ MB


In [7]:
df_train.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Holiday,Discount,#Order,Sales
Date,,,,,,,,,
2018-01-01,T1000001,1,S1,L3,R1,1,Yes,9,7011.84
2018-01-01,T1000002,253,S4,L2,R1,1,Yes,60,51789.12
2018-01-01,T1000003,252,S3,L2,R1,1,Yes,42,36868.20
2018-01-01,T1000004,251,S2,L3,R1,1,Yes,23,19715.16
2018-01-01,T1000005,250,S2,L3,R4,1,Yes,62,45614.52


In [8]:
 df_train[df_train['Sales']  <1].count()

ID               19
Store_id         19
Store_Type       19
Location_Type    19
Region_Code      19
Holiday          19
Discount         19
#Order           19
Sales            19
dtype: int64

In [9]:
df_train.shape

(188340, 9)

In [10]:
df_train = df_train[df_train.Sales != 0]

In [11]:
df_train.shape

(188321, 9)

In [12]:
# plt.figure(figsize = (20,10))
# sns.lineplot(x= 'Date', y= 'Sales', hue = 'Holiday', data = df_train)

In [13]:
# plt.figure(figsize = (20,10))
# sns.lineplot(x= 'Date', y= 'Sales', hue = 'Discount', data = df_train)

In [14]:
cat_data_regt = pd.get_dummies(df_train["Region_Code"], drop_first=True)

cat_data_stot = pd.get_dummies(df_train["Store_Type"], drop_first=True)

cat_data_loct = pd.get_dummies(df_train["Location_Type"], drop_first=True)

cat_data_dist = pd.get_dummies(df_train["Discount"], drop_first=True)

In [15]:
cat_data_reg = pd.get_dummies(df_test["Region_Code"], drop_first=True)
cat_data_sto = pd.get_dummies(df_test["Store_Type"], drop_first=True)
cat_data_loc = pd.get_dummies(df_test["Location_Type"], drop_first=True)
cat_data_dis = pd.get_dummies(df_test["Discount"], drop_first=True)

In [16]:
df_train.corr(method ='spearman')

,Store_id,Holiday,#Order,Sales
Store_id,1.000000,0.000050,0.007633,-0.011444
Holiday,0.000050,1.000000,-0.146408,-0.158934
#Order,0.007633,-0.146408,1.000000,0.937727
Sales,-0.011444,-0.158934,0.937727,1.000000


In [17]:
train_data = df_train.drop(['ID','Store_Type','Location_Type','Region_Code','Discount','#Order'],axis = 1)
test_data = df_test.drop(['ID','Store_Type','Location_Type','Region_Code','Discount'],axis = 1)

In [18]:
train_data.head(1)

,Store_id,Holiday,Sales
Date,,,
2018-01-01,1,1,7011.84


In [19]:
test_data.head(1)

,Store_id,Holiday
Date,,
2019-06-01,171,0


In [20]:
train = pd.concat([train_data,cat_data_regt,cat_data_stot,cat_data_loct,cat_data_dist], axis = 1)

test = pd.concat([test_data,cat_data_reg,cat_data_sto,cat_data_loc,cat_data_dis], axis = 1)


In [21]:
train.head(1)

,Store_id,Holiday,Sales,R2,R3,R4,S2,S3,S4,L2,L3,L4,L5,Yes
Date,,,,,,,,,,,,,,
2018-01-01,1,1,7011.84,0,0,0,0,0,0,0,1,0,0,1


In [22]:
test.head(1)

,Store_id,Holiday,R2,R3,R4,S2,S3,S4,L2,L3,L4,L5,Yes
Date,,,,,,,,,,,,,
2019-06-01,171,0,0,1,0,0,0,1,1,0,0,0,0


In [23]:
train.head(1)

,Store_id,Holiday,Sales,R2,R3,R4,S2,S3,S4,L2,L3,L4,L5,Yes
Date,,,,,,,,,,,,,,
2018-01-01,1,1,7011.84,0,0,0,0,0,0,0,1,0,0,1


In [24]:
X = train.drop(['Sales'],axis = 1)
y= train['Sales']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

<IPython.core.display.Javascript object>

# lightgbm

In [26]:
import lightgbm as ltb
model = ltb.LGBMRegressor()
model.fit(X_train, y_train)


LGBMRegressor()

In [27]:
model.score(X_test,y_test)

0.6401567493412963

# XGBRegressor

In [28]:
from xgboost import XGBRegressor

xgb_reg_model = XGBRegressor() #max_depth=4,n_estimators = 380,learning_rate = 0.02)

xgb_reg_model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
xgb_reg_model.score(X_test, y_test)

0.6492058715212969

In [30]:
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# from sklearn.preprocessing import Imputer

In [31]:
#XGBoost hyper-parameter tuning
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [32]:
hyperParameterTuning(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 200,
 'objective': 'reg:squarederror',
 'subsample': 0.7}

In [34]:
xgb_model = XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.1,
        max_depth = 10,
        min_child_weight = 1,
        n_estimators = 200,
        subsample = 0.7)

xgb_model.fit(X_train, y_train, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [35]:
model.score(X_test,y_test)

0.6401567493412963

# LinearRegression

In [26]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [27]:
model.score(X_test, y_test)

0.551891539095642

In [28]:
test_pred = model.predict(test)


In [30]:
sub = pd.read_csv('SAMPLE.csv')

In [31]:
sub.head()

,ID,Sales
0,T1188341,42275
1,T1188342,42275
2,T1188343,42275
3,T1188344,42275
4,T1188345,42275


In [32]:
sub['Sales'] = test_pred

In [33]:
# sub.head(1)

In [34]:
sub.to_csv('submissionFinal.csv', index=False)

# AdaBoostRegressor

In [116]:
from sklearn.ensemble import AdaBoostRegressor

In [117]:
regr = AdaBoostRegressor(random_state=0, n_estimators=50,learning_rate=0.1)
regr.fit(X_train, y_train)

AdaBoostRegressor(learning_rate=0.1, random_state=0)

In [118]:
regr.score(X_test, y_test)

0.4887337610549234

In [131]:
from sklearn.tree import DecisionTreeRegressor
regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=50),n_estimators=400, random_state=0)
regr_2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=50),
                  n_estimators=400, random_state=0)

In [132]:
regr_2.score(X_test, y_test)

0.5850437148628553

#  GradientBoostingRegressor

In [52]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)

GradientBoostingRegressor(random_state=0)

In [58]:
reg.score(X_test, y_test)

0.5993168697476088

# HistGradientBoostingRegressor

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingRegressor

est = HistGradientBoostingRegressor()
est.fit(X_train, y_train)

In [ ]:
est.score(X_test, y_test)

# ARDRegression

In [61]:
from sklearn import linear_model
clf = linear_model.ARDRegression(compute_score=True)
clf.fit(X_train, y_train)

ARDRegression(compute_score=True)

In [62]:
clf.score(X_test, y_test)

0.551885989494291

# BayesianRidge

In [63]:
from sklearn import linear_model
clf = linear_model.BayesianRidge()
clf.fit(X_train, y_train)

BayesianRidge()

In [64]:
clf.score(X_test, y_test)

0.5518917663730772

# BaggingRegressor

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor


regr = BaggingRegressor(base_estimator=SVR(),n_estimators=10, random_state=0)
regr.fit(X_train,y_train)


In [ ]:
regr.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
estimators = [("Tree", DecisionTreeRegressor()),
              ("Bagging(Tree)", BaggingRegressor(DecisionTreeRegressor()))]

estimator.fit(X_train,y_train)

In [ ]:
estimator.score(X_test, y_test)

# ExtraTreesRegressor

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
reg = ExtraTreesRegressor(n_estimators=100, random_state=0)
reg.fit(X_train,y_train)

In [ ]:
reg.score(X_test, y_test)

# RandomForestRegressor

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [39]:
reg = RandomForestRegressor(n_jobs=-1, n_estimators=400,bootstrap= True,max_depth=90,max_features='auto',min_samples_leaf= 4,min_samples_split = 10)
  

reg.fit(X_train, y_train)
  
y_pred = reg.predict(X_test)
  

In [40]:
reg.score(X_test, y_test)

0.6478800141399533

In [41]:
test_pred = reg.predict(test)

In [42]:
sub['Sales'] = test_pred

In [43]:
sub.to_csv('submission_RFR.csv', index=False)

In [44]:
# from sklearn.model_selection import RandomizedSearchCV
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# print(random_grid)

In [45]:
# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X_train, y_train)

In [46]:
# rf_random.best_params_

# Lasso

In [47]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)

In [48]:
clf.fit(X_train, y_train)

Lasso(alpha=0.1)

In [49]:
y_pred = clf.predict(X_test)

In [50]:
clf.score(X_test, y_test)

0.5518912537934446

# DecisionTreeRegressor

In [51]:
from sklearn.tree import DecisionTreeRegressor
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_2.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5)

In [52]:
y_pred = regr_2.predict(X_test)
regr_2.score(X_test, y_test)

0.5388586508644826

# KNeighborsRegressor

In [53]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=2)

In [54]:
y_pred = neigh.predict(X_test)
neigh.score(X_test, y_test)

0.46816363301550135

# Support Vector Machines (SVM)

In [55]:
from sklearn import svm

In [ ]:
regr = svm.SVR()
regr.fit(X_train, y_train)

In [ ]:
# y_pred =regr.predict(X_test)
regr.score(X_test, y_test)

# Neural Network Regression

In [67]:
from sklearn.neural_network import MLPRegressor

In [68]:
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
regr.score(X_test, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


0.566339993105212

# VotingRegressor

In [69]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

r1 = LinearRegression()
r2 = RandomForestRegressor(n_estimators=10, random_state=1)

er = VotingRegressor([('lr', r1), ('rf', r2)])

er.fit(X_train, y_train)

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rf',
                             RandomForestRegressor(n_estimators=10,
                                                   random_state=1))])

In [70]:
er.score(X_test, y_test)

0.6280083932976954

# StackingRegressor

In [71]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

estimators = [('lr', RidgeCV()),('svr', LinearSVR(random_state=42))]
reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))


reg.fit(X_train, y_train)

StackingRegressor(estimators=[('lr', RidgeCV(alphas=array([ 0.1,  1. , 10. ]))),
                              ('svr', LinearSVR(random_state=42))],
                  final_estimator=RandomForestRegressor(n_estimators=10,
                                                        random_state=42))

In [72]:
reg.score(X_test, y_test)

0.48472500124210227